In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [ ]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import resnext as unet
from malaya_speech.utils import tf_featurization
import malaya_speech.augmentation.waveform as augmentation
import IPython.display as ipd
import numpy as np
from malaya_speech.utils.tf_featurization import separation_exponent, EPSILON

In [ ]:
# !pip3 install museval
import museval

In [ ]:
class Model:
    def __init__(self, size = 2):
        self.X = tf.placeholder(tf.float32, (None))
        self.Y = tf.placeholder(tf.float32, (size, None))
        
        stft_X, D_X = tf_featurization.get_stft(self.X)
        
        self.stft = []
        for i in range(size):
            self.stft.append(tf_featurization.get_stft(self.Y[i]))
        
        self.outputs = []
        for i in range(size):
            with tf.variable_scope(f'model_{i}'):
                self.outputs.append(unet.Model(D_X, num_layers = 6).logits)
        
        self.loss = []
        for i in range(size):
            self.loss.append(
                tf.reduce_mean(tf.abs(self.outputs[i] - self.stft[i][1]))
            )

        self.cost = tf.reduce_sum(self.loss)
        
        output_sum = tf.reduce_sum([o ** separation_exponent for o in self.outputs], axis=0) + EPSILON
        
        self.istft = []
        for no, D in enumerate(self.outputs):

            instrument_mask = (D ** separation_exponent + (EPSILON / size)) / output_sum
            instrument_mask = tf_featurization.extend_mask(instrument_mask)
            old_shape = tf.shape(instrument_mask)
            new_shape = tf.concat(
                [[old_shape[0] * old_shape[1]], old_shape[2:]],
                axis=0)
            instrument_mask = tf.reshape(instrument_mask, new_shape)

            instrument_mask = instrument_mask[:tf.shape(stft_X)[0]]
            masked_stft = tf.cast(instrument_mask, dtype=tf.complex64) * stft_X
            self.istft.append(tf_featurization.istft(masked_stft, self.X)[:,0])
        
        for i in range(size):
            tf.identity(self.istft[i], name = f'logits_{i}')

In [ ]:
tf.compat.v1.reset_default_graph()
model = Model()
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
var_lists = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, 'noise-reduction-resnext/model.ckpt-500000')

In [ ]:
import pickle

with open('test-set-noise-reduction.pkl', 'rb') as fopen:
    results = pickle.load(fopen)

In [ ]:
results[0]

In [ ]:
sess.run([model.cost, model.loss], feed_dict = {model.X: results[0][0],
                                               model.Y: results[0][1:]})

In [ ]:
from tqdm import tqdm

total, voice, noise = [], [], []

SDR, ISR, SAR = [], [], []

for i in tqdm(range(len(results))):
    c, l = sess.run([model.cost, model.loss], feed_dict = {model.X: results[i][0],
                                               model.Y: results[i][1:]})
    total.append(c)
    voice.append(l[0])
    noise.append(l[1])
    
    y_ = sess.run(model.istft, feed_dict = {model.X: results[i][0]})[0]
    sdr, isr, _, sar = museval.evaluate(np.reshape(results[i][1], (1, -1)), 
                                        np.reshape(y_, (1, -1)))
    SDR.append(np.nanmean(sdr))
    ISR.append(np.nanmean(isr))
    SAR.append(np.nanmean(sar))

In [ ]:
np.mean(total)

In [ ]:
np.mean(voice)

In [ ]:
np.mean(noise)

In [ ]:
np.mean(SDR)

In [ ]:
np.mean(ISR)

In [ ]:
np.mean(SAR)

In [ ]:
def read_wav(f):
    return malaya_speech.load(f, sr = 44100)

y = read_wav('2x5%20Ep%2010.wav')[0]
# n = read_wav(random.choice(noises))[0]
sr = 44100

In [ ]:
istft = sess.run(model.istft, feed_dict = {model.X: y})

In [ ]:
import IPython.display as ipd

In [ ]:
ipd.Audio(y[:sr * 15], rate = sr)

In [ ]:
ipd.Audio(istft[0][:sr * 15], rate = sr)

In [ ]:
saver = tf.train.Saver()
saver.save(sess, 'noise-reduction-resnext-output/model.ckpt')

In [ ]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'Assign' not in n.name
    ]
)
strings.split(',')

In [ ]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [ ]:
freeze_graph('noise-reduction-resnext-output', strings)

In [ ]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [ ]:
instruments = ['voice', 'noise']

In [ ]:
g = load_graph('noise-reduction-resnext-output/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')

logits = {instrument: g.get_tensor_by_name(f'import/logits_{no}:0') for no, instrument in enumerate(instruments)}

In [ ]:
test_sess = tf.InteractiveSession(graph = g)

In [ ]:
l = test_sess.run(logits, feed_dict = {x: y})

In [ ]:
ipd.Audio(l['voice'][:sr * 15], rate = sr)

In [ ]:
from tensorflow.tools.graph_transforms import TransformGraph

pb = 'noise-reduction-resnext-output/frozen_model.pb'

transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-1024, fallback_max=1024)',
             'strip_unused_nodes',
             'sort_by_execution_order']

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                           ['Placeholder'],
                                           [f'logits_{no}' for no in range(len(instruments))], transforms)
    
with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

In [ ]:
g = load_graph('noise-reduction-resnext-output/frozen_model.pb.quantized')
x = g.get_tensor_by_name('import/Placeholder:0')

logits = {instrument: g.get_tensor_by_name(f'import/logits_{no}:0') for no, instrument in enumerate(instruments)}

In [ ]:
test_sess = tf.InteractiveSession(graph = g)

In [ ]:
l = test_sess.run(logits, feed_dict = {x: y})

In [ ]:
ipd.Audio(l['voice'][:sr * 15], rate = sr)